# Castaways reactions
> How did contestants respond after their torch was snuffed? This script fetches the growing log and scores the acknowlegement of his/her tribe.rt    

#### Load Python tools and Jupyter config

In [1]:
import pandas as pd
import jupyter_black

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [3]:
# Output paths
csv_output_path = "../data/raw/other/vote_offs/log.csv"
json_output_path = "../data/raw/other/vote_offs/log.json"

---

## Read data

#### Get data entry table

In [4]:
# Define URL for Google Sheets CSV
# https://docs.google.com/spreadsheets/d/1nys0mCWArUCtPKYIVBrbjmv7eAWkmOce4cBlyHm8b0c/edit?gid=120496879#gid=120496879
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQfmJJvr0TdW6fZ-wDdTadFDY5wTm2S6cnr9-n2D3cS6-eRGbIySzVokY97bON66_BXDIc0jnSjch82/pub?gid=120496879&single=true&output=csv"

# Read CSV from URL
data_entry = pd.read_csv(url)

In [ ]:
# clean castaways table
vote_offs_lookup = pd.read_csv("../data/processed/survivor_all_voteoffs.csv")

In [17]:
vote_offs_lookup.query("season == 42")

,season,episode,voted_out,voted_out_id,vote
623,42,1,Zach,US0626,1
624,42,2,Marya,US0618,2
625,42,3,Jenny,US0614,3
626,42,4,Swati,US0624,4
627,42,5,Daniel,US0610,5
628,42,7,Lydia,US0617,6
629,42,8,Chanelle,US0609,7
630,42,9,Rocksroy,US0622,8
631,42,9,Tori,US0625,9
632,42,10,Hai,US0612,10


In [6]:
# Assertion to ensure the lengths are the same
assert len(data_entry) == len(
    vote_offs_lookup
), f"Assertion failed: Lengths do not match (data_entry: {data_entry_rows}, vote_offs_lookup: {vote_offs_rows})"

print("Lengths match successfully!")

Lengths match successfully!


In [7]:
# Fix data error
vote_offs_lookup.loc[
    (vote_offs_lookup["season"] == 46) & (vote_offs_lookup["episode"] == 8), "voted_out"
] = "Tevin"

In [8]:
vote_offs_lookup.loc[
    (vote_offs_lookup["season"] == 46) & (vote_offs_lookup["episode"] == 8),
    "voted_out_id",
] = "US0694"

In [9]:
# Merge them together to add ids to Google Sheet
merged = pd.merge(
    data_entry, vote_offs_lookup, on=["season", "vote"], how="right", indicator=True
)

In [10]:
merged["match"] = merged["castaway"] == merged["voted_out"]

In [11]:
assert merged[
    "match"
].all(), "Assertion failed: There are mismatched rows in the DataFrame."

print("All rows matched successfully!")

All rows matched successfully!


In [12]:
# Define acknowledgment columns
ack_columns = ["ack_gesture", "ack_speak", "ack_look", "ack_smile"]

In [13]:
# Calculate acknowledgment score as the count of True values in the acknowledgment columns
merged["ack_score"] = merged[ack_columns].sum(axis=1)

In [14]:
# Save to CSV
merged.to_csv(csv_output_path, index=False)

In [15]:
# Save to JSON
merged.to_json(json_output_path, orient="records", indent=4)